Get necessary modules, and define constants

In [ ]:
import os, cv2, sys
import numpy as np
from scipy import sqrt, pi, arctan2, cos, sin
from scipy.ndimage import uniform_filter
from sklearn import svm, metrics, tree, ensemble, linear_model
from sklearn.model_selection import cross_val_score

# The dimensions to resize video frames to before calculating HoF
FIXED_WIDTH = 128
FIXED_HEIGHT = 128

# Property useful for finding length (in frames) of a video.
# Renamed here because the way to access it can differ between machines, so we can change it easily
FRAME_LENGTH_PROP = cv2.CAP_PROP_FRAME_COUNT

# Threshold ratio for segmentation. Segments longer than this * length of smallest video are not discarded
SEG_THRESHOLD = 0.5

# Ratio of training data to testing data
#TRAIN_RATIO = 0.7

# level parameter for pyramiding HoF
LEVEL = 3

Do some setup necessary for all/most descriptors

In [ ]:
# Directory and files of the first action
ACT1_DIR = "./hof/run/"
ACT1_FILES = os.listdir(ACT1_DIR)
ACT1_FILES = [ACT1_DIR + f for f in ACT1_FILES]

# Directory and files of the second action
ACT2_DIR = "./hof/walk/"
ACT2_FILES = os.listdir(ACT2_DIR)
ACT2_FILES = [ACT2_DIR + f for f in ACT2_FILES]

# Directory and files of the third action
ACT3_DIR = "./hof/walk/"
ACT3_FILES = os.listdir(ACT3_DIR)
ACT3_FILES = [ACT3_DIR + f for f in ACT3_FILES]

# Directory and files of the fourth action
ACT4_DIR = "./hof/walk/"
ACT4_FILES = os.listdir(ACT4_DIR)
ACT4_FILES = [ACT4_DIR + f for f in ACT4_FILES]

# Directory and files of the fifth action
ACT5_DIR = "./hof/walk/"
ACT5_FILES = os.listdir(ACT5_DIR)
ACT5_FILES = [ACT5_DIR + f for f in ACT5_FILES]

# Use equal number of data from each class, setting a cap at a total of 464 files
nc = $min(len(ACT1_FILES), len(ACT2_FILES), len(ACT3_FILES), len(ACT4_FILES), len(ACT5_FILES))
nc = nc if (nc < 92) else 92
print "nc:", nc
ACT1_FILES = ACT1_FILES[0:nc]
ACT2_FILES = ACT2_FILES[0:nc]
ACT3_FILES = ACT3_FILES[0:nc]
ACT4_FILES = ACT4_FILES[0:nc]
ACT5_FILES = ACT5_FILES[0:nc]

#offset = int(np.floor(nc*TRAIN_RATIO))
#print "offset:", offset

# Split test and training at ratio
train_files = ACT1_FILES[0:nc] + ACT2_FILES[0:nc] + ACT3_FILES[0:nc] + ACT4_FILES[0:nc] + ACT5_FILES[0:nc]
#test_files = ACT1_FILES[offset:nc] + ACT2_FILES[offset:nc] + ACT3_FILES[offset:nc] + ACT4_FILES[offset:nc] + ACT5_FILES[offset:nc]

# Put the labels in vectors
train_labels = np.zeros(nc*5, int)
train_labels[0:nc] = 1
train_labels[nc:nc*2] = 2
train_labels[nc*2:nc*3] = 3
train_labels[nc*3:nc*4] = 4
train_labels[nc*4:nc*5] = 5

#test_len = nc-offset
#test_labels = np.zeros(test_len*5, int)
#test_labels[0:test_len] = 1
#test_labels[test_len:test_len*2] = 2
#test_labels[test_len*2:test_len*3] = 3
#test_labels[test_len*3:test_len*4] = 4
#test_labels[test_len*4:test_len*5] = 5

print "train files:", len(train_files)
print "train labels:", len(train_labels)
#print "test files:", len(test_files)
#print "test labels:", len(test_labels)

Define a method and parameters for normalizing frames, by transforming them to fixed dimensions and putting them in grayscale

In [ ]:
def normalizeFrame(frame_original):
    frame_gray = cv2.cvtColor(frame_original,cv2.COLOR_BGR2GRAY)
    frame_gray_resized = cv2.resize(frame_gray, (FIXED_WIDTH, FIXED_HEIGHT))
    return frame_gray_resized

Define optical flow for this project. Uses the Farneback method for calculation.

In [ ]:
# Gets the optical flow [<dx,dy>] from two frames
def getOpticalFlow(imPrev, imNew):
    flow = cv2.calcOpticalFlowFarneback(imPrev, imNew, flow=None, pyr_scale=.5, levels=3, winsize=9, 
                                        iterations=1, poly_n=3, poly_sigma=1.1, 
                                        flags=cv2.OPTFLOW_FARNEBACK_GAUSSIAN)
    return flow

Define HoF, which takes as parameters the optical flow between two frames, and the numbers of pixels and cells per histogram block

In [ ]:
# Compute the Histogram of Optical Flow (HoF) from the given optical flow
def hof(flow, pixels_per_cell, cells_per_block):

    MOTION_THRESHOLD=1.0
    NUM_ORIENTATIONS = 9

    flow = np.atleast_2d(flow)

    if flow.dtype.kind == 'u':
        flow = flow.astype('float')

    # Create NParray filled with zeroes
    gx = np.zeros(flow.shape[:2])
    gy = np.zeros(flow.shape[:2])

    # Find Gx and Gy from the flow field
    gx = flow[:,:,1]
    gy = flow[:,:,0]


    magnitude = sqrt(gx**2 + gy**2)

    orientation = arctan2(gy, gx)
    orientation = orientation * (180 / pi) % 180

    shape_y, shape_x = flow.shape[:2]

    x_pixels, y_pixels = pixels_per_cell
    y_blocks, y_blocks = cells_per_block

    cells_x = int(np.floor(shape_x // x_pixels))
    cells_y = int(np.floor(shape_y // y_pixels))

    orientation_bins = np.zeros((cells_y, cells_x, NUM_ORIENTATIONS))

    subsample = np.index_exp[y_pixels / 2:y_pixels * cells_y:y_pixels, x_pixels / 2:x_pixels * cells_x:x_pixels]

    for i in range(NUM_ORIENTATIONS-1):

        predicate_less =  orientation < 180 / NUM_ORIENTATIONS * (i + 1)
        predicate_greater = orientation >= 180 / NUM_ORIENTATIONS * i

        # Filter by orientation

        orientation_ = np.where(predicate_less, orientation, -1)
        orientation_ = np.where(predicate_greater, orientation_, -1)

        predicate_motion = (orientation_ > -1) * (magnitude > MOTION_THRESHOLD)

        _magnitude = np.where(predicate_motion, magnitude, 0)

        # Filter by magnitude
        filtered = uniform_filter(_magnitude, size=(y_pixels, x_pixels))
        orientation_bins[:, :, i] = filtered[subsample]

    _magnitude = np.where(magnitude <= MOTION_THRESHOLD, magnitude, 0)

    filtered = uniform_filter(_magnitude, size=(y_pixels, x_pixels))
    orientation_bins[:, :, -1] = filtered[subsample]

    num_blocks_x = (cells_x - y_blocks) + 1
    num_blocks_y = (cells_y - y_blocks) + 1

    # Create matrix
    result_blocks = np.zeros((num_blocks_y, num_blocks_x,
                                  y_blocks, y_blocks, orientations))

    # Fill matrix
    for x in range(num_blocks_x):
        for y in range(num_blocks_y):
            block = orientation_bins[y:y+y_blocks, x:x+y_blocks, :]
            denom = sqrt(block.sum()**2 + 1e-5)
            result_blocks[y, x, :] = block / denom

    return result_blocks.ravel()


Define a method for calculating HoF between two frames

In [ ]:
# get the Histogram of Optical Flow from two images
def getHoF(frame1, frame2):
    flow = getOpticalFlow(frame1, frame2)
    return hof(flow, pixels_per_cell=(20,20), cells_per_block=(5,5))

And another for calculating the HoF feature vector for an entire video

In [ ]:
# get the Histogram of Optical Flows of a video grouped sequentially in a 1D array
def getSequentialHoF(video_path):
    hofs = []
    cap = cv2.VideoCapture(video_path)
    ret1, frame1 = cap.read()
    frame1 = normalizeFrame(frame1)
    while(cap.isOpened()):
        ret2, frame2 = cap.read()
        if ret2 == True:
            frame2 = normalizeFrame(frame2)
            hof_array = getHoF(frame1, frame2)
            hofs = np.concatenate((hofs, hof_array),axis=0)
            frame1 = frame2
        else:
            break
    return hofs

So far, we have all the methods necessary for a base HoF descriptor; however, having training/testing videos of different lengths means our feature vectors will have different lengths (not good for classifiers).

One approach to remedy this is to pad shorter vectors with zeros to the length of the largest vector. We define two methods to help us with this:

In [ ]:
# Find the length of the largest row in training set and testing set
def maxRow(test, train):
    return max(np.array([len(i) for i in train]).max(),
               np.array([len(i) for i in test]).max())

# Pad each row of the 2D array, with 0, to a specified width
def numpy_fillna(data, width):
    # Get lengths of each row of data
    lens = np.array([len(i) for i in data])

    # Mask of valid places in each row
    mask = np.arange(width) < lens[:,None]

    # Setup output array and put elements from data into masked positions
    out = np.zeros(mask.shape, dtype=data.dtype)
    out[mask] = np.concatenate(data)
    return out

Finally, we define a method which gets all the feature vectors we need, ensuring they are padded to the right length

In [ ]:
def getFeatures_Baseline(train_files, train_labels):#, test_files, test_labels):
    # Make feature vectors
    train = [getSequentialHoF(p) for p in train_files]
    #test = [getSequentialHoF(p) for p in test_files]
    
    # Pad them to the max video width
    max_width = maxRow(test, train);
    train_pad = numpy_fillna(np.array(train), max_width)
    #test_pad = numpy_fillna(np.array(test), max_width)
    
    return train_pad, train_labels#, test_pad, test_labels
    

This approach works, but may not give us the best accuracy since it introduces noise, especially for shorter videos. A different approach we take is to trim the videos to the length of the smallest video and then apply HoF, using frames only from the middle of each video.

We define a method to find the length of the shortest video in the video directory

In [ ]:
# Determine the length in frames of the shortest video in the provided dataset
def shortest(data_dir):
    # get list of files in the directory. directory should be flat with only video files in it
    files = os.listdir(data_dir)
    
    # Find the length of the shortest video (in frames)
    shortestLen = sys.maxint
    for i in range(len(files)):
        cap = cv2.VideoCapture(data_dir+files[i])
        length = int(cap.get(FRAME_LENGTH_PROP))
        if length < shortestLen:
            shortestLen = length
            
        cap.release()
        
    return shortestLen

And a method which gets the HoF feature vector from the middle of a video, using the amount of frames determined by the above method (passed as param). This is the same as the base sequential HoF method, modified to get frames from the middle.

In [ ]:
# Get the Histogram of Optical Flows of a video grouped sequentially in a 1D array
# Use only the specified amount of frames, from the middle of the video
def getSequentialHoFMiddle(video_path, frames):
    hofs = []
    cap = cv2.VideoCapture(video_path)
    length = int(cap.get(FRAME_LENGTH_PROP))
    startIdx = ((length - 1) - frames)/2
    if startIdx < 1:
        startIdx = 0
        
    # skip through beginning unneeded frames
    frameNum = 0
    while (frameNum < startIdx):
        cap.grab()
        frameNum += 1
    frameNum = 0
    
    # Calculate HoF from necessary frames
    ret1, frame1 = cap.read()
    frame1 = normalizeFrame(frame1)
    while(frameNum < frames-2):
        ret2, frame2 = cap.read()
        if ret2 == True:
            frame2 = normalizeFrame(frame2)
            hof_array = getHoF(frame1, frame2)
            hofs = np.concatenate((hofs, hof_array),axis=0)
            frame1 = frame2
            frameNum += 1
        else:
            break
    
    cap.release()
    return hofs

Finally, we define a method which gets all the feature vectors we need, ensuring they are made from trimmed video middles

In [ ]:
def getFeatures_Trimmed(train_files, train_labels):#, test_files, test_labels):
    numFrames = min(shortest(ACT1_DIR), shortest(ACT2_DIR))
    train = [getSequentialHoFMiddle(p, numFrames) for p in train_files]
    #test = [getSequentialHoFMiddle(p, numFrames) for p in test_files]
    
    return np.array(train), train_labels#, np.array(test), test_labels

This gives us feature vectors with less noise than the baseline method, but can also eliminate a lot of useful information from longer videos. Another method which preserves most of this information while also ensuring uniform-length vectors, is to break each video into fixed-length segments, where each segment is the length of the shortest video in the set.

We define another method for calculating HoF feature vectors on each segment of a video.

In [ ]:
def getSequentialHoFSegments(video_path, label, frames):
    seg_hofs = []
    hofs = []
    cap = cv2.VideoCapture(video_path)
    length = int(cap.get(FRAME_LENGTH_PROP))
    
    ret1, frame1 = cap.read()
    frame1 = normalizeFrame(frame1)
    while(cap.isOpened()):
        hofs = []
        for i in range(frames-1):
            ret2, frame2 = cap.read()
            if ret2 == True:
                frame2 = normalizeFrame(frame2)
                hof_array = getHoF(frame1, frame2)
                hofs = np.concatenate((hofs, hof_array),axis=0)
                frame1 = frame2
            else:
                cap.release()
                break
        seg_hofs.append(hofs)
    
    seg_labels = np.full(len(seg_hofs), label)
    return seg_hofs, seg_labels

The above method can produce one segment per video which is shorter than the minimum length (from the end). We could pad these, but for short segments this would introduce a lot of noise. We define a threshold (is a constant defined at top) length for these videos, pad those above the threshold, and discard segments which are too short.

In [ ]:
def getFeatures_Segmented(train_files, train_labels):#, test_files, test_labels):
    numFrames = min(shortest(ACT1_DIR), shortest(ACT2_DIR))
    train_result = [a for t,l in zip(train_files, train_labels) for a in getSequentialHoFSegments(t, l, numFrames)]
    #test_result = [a for t,l in zip(test_files, test_labels) for a in getSequentialHoFSegments(t, l, numFrames)]
    train = [y for x in train_result[::2] for y in x]
    new_train_labels = np.concatenate(train_result[1::2])
    #test = [y for x in test_result[::2] for y in x]
    #new_test_labels = np.concatenate(test_result[1::2])
    train_threshold = int(max([len(x) for x in train]) * SEG_THRESHOLD)
    #test_threshold = int(max([len(x) for x in test]) * SEG_THRESHOLD)
    train_trimmed = [x for x,y in zip(train,new_train_labels) if len(x) > train_threshold]
    train_labels_trimmed = [y for x,y in zip(train,new_train_labels) if len(x) > train_threshold]
    #test_trimmed = [x for x,y in zip(test,new_test_labels) if len(x) > test_threshold]
    #test_labels_trimmed = [y for x,y in zip(test,new_test_labels) if len(x) > test_threshold]
    max_width = max(np.array([len(i) for i in test_trimmed]).max(),np.array([len(i) for i in test_trimmed]).max())
    train_pad = numpy_fillna(np.array(train_trimmed), max_width)
    #test_pad = numpy_fillna(np.array(test_trimmed), max_width)
    
    return train_pad, train_labels_trimmed#, test_pad, test_labels_trimmed

We've now defined how to get features for HoF, using the padding, trimming, and segmentation methods of ensuring feature vectors are the same shape. Additionally, we want to make HoF feature vectors which use gaussian pyramiding in order to help reduce noise factors in videos.

Define a method for getting a pyramid for an image

In [ ]:
def getPyramid(img, level):
    p = []
    p.append(img)
    for i in range(level):
        p.append(cv2.pyrDown(p[len(p)-1]))
    return p

Define a method for getting pyramid HoF between two frames

In [ ]:
def pyramidHof(frame1, frame2, level, getHofFunc, getPyramidFunc, getOpticalFlowFunc):
    if frame1.shape != frame2.shape:
        raise ValueError('frame1 and frame2 should have identical dimensions')
    pyramid1 = getPyramidFunc(frame1, level)
    pyramid2 = getPyramidFunc(frame2, level)
    pyramidHoF = []
    for i in range(level):
        subframe1 = pyramid1[i]
        subframe2 = pyramid2[i]
        subflow = getOpticalFlowFunc(subframe1, subframe2)
        subhof = getHofFunc(subflow)
        pyramidHoF = np.append(pyramidHoF, subhof)
    return pyramidHoF

Define a method for determining pyramid HoF on whole video

In [ ]:
def getPyramidHof(video_path, level, normalizeFrameFunc=normalizeFrame, getHofFunc=hof, getPyramidFunc=getPyramid, getOpticalFlowFunc=getOpticalFlow):
    cap = cv2.VideoCapture(video_path)
    vid_hof = []
    ret1, frame1 = cap.read()
    frame1 = normalizeFrameFunc(frame1)
    while(cap.isOpened()):
        ret2, frame2 = cap.read()
        if ret2 == True:
            frame2 = normalizeFrameFunc(frame2)
            biframe_hof = pyramidHof(frame1, frame2, level, getHofFunc, getPyramidFunc, getOpticalFlowFunc)
            vid_hof = np.append(vid_hof, biframe_hof)
            frame1 = frame2
        else:
            break
    cap.release()
    return vid_hof

Define a method for getting pyramid HoF feature vectors

In [ ]:
def getFeatures_Pyramid_Baseline(train_files, train_labels):#, test_files, test_labels):
    # make feature vectors
    train = [getPyramidHof(p, LEVEL) for p in train_files]
    #test = [getPyramidHof(p, LEVEL) for p in test_files]
    
    # Pad them to the max video width
    max_width = maxRow(test, train);
    train_pad = numpy_fillna(np.array(train), max_width)
    #test_pad = numpy_fillna(np.array(test), max_width)
    
    return train_pad, train_labels#, test_pad, test_labels

Now we want to repeat what we did before with trimming for pyramid HoF

In [ ]:
def getPyramidHofMiddle(video_path, frames, level, normalizeFrameFunc=normalizeFrame, getHofFunc=hof, getPyramidFunc=getPyramid, getOpticalFlowFunc=getOpticalFlow):
    cap = cv2.VideoCapture(video_path)
    vid_hof = []
    
    length = int(cap.get(FRAME_LENGTH_PROP))
    startIdx = ((length - 1) - frames)/2
    if startIdx < 1:
        startIdx = 0
    # skip through beginning unneeded frames
    frameNum = 0
    while (frameNum < startIdx):
        cap.grab()
        frameNum += 1
    frameNum = 0
    
    ret1, frame1 = cap.read()
    frame1 = normalizeFrameFunc(frame1)
    while(frameNum < frames-2):
        ret2, frame2 = cap.read()
        if ret2 == True:
            frame2 = normalizeFrameFunc(frame2)
            biframe_hof = pyramidHof(frame1, frame2, level, getHofFunc, getPyramidFunc, getOpticalFlowFunc)
            vid_hof = np.append(vid_hof, biframe_hof)
            frame1 = frame2
            frameNum += 1
        else:
            break
    cap.release()
    return vid_hof

def getFeatures_Pyramid_Trimmed(train_files, train_labels):#, test_files, test_labels):
    numFrames = min(shortest(ACT1_DIR), shortest(ACT2_DIR))
    train = [getPyramidHofMiddle(p, numFrames, LEVEL) for p in train_files]
    #test = [getPyramidHofMiddle(p, numFrames, LEVEL) for p in test_files]
    
    return np.array(train), train_labels#, np.array(test), test_labels

...As well as for segmentation

In [ ]:
def getPyramidHoFSegments(video_path, label, frames, level, normalizeFrameFunc=normalizeFrame, getHofFunc=hof, getPyramidFunc=getPyramid, getOpticalFlowFunc=getOpticalFlow):
    seg_hofs = []
    hofs = []
    cap = cv2.VideoCapture(video_path)
    length = int(cap.get(FRAME_LENGTH_PROP))
    
    ret1, frame1 = cap.read()
    frame1 = normalizeFrame(frame1)
    while(cap.isOpened()):
        hofs = []
        for i in range(frames-1):
            ret2, frame2 = cap.read()
            if ret2 == True:
                frame2 = normalizeFrameFunc(frame2)
                biframe_hof = pyramidHof(frame1, frame2, level, getHofFunc, getPyramidFunc, getOpticalFlowFunc)
                hofs = np.concatenate((hofs, biframe_hof),axis=0)
                frame1 = frame2
            else:
                cap.release()
                break
        seg_hofs.append(hofs)
    
    seg_labels = np.full(len(seg_hofs), label)
    return seg_hofs, seg_labels

def getFeatures_Pyramid_Segmented(train_files, train_labels):#, test_files, test_labels):
    numFrames = min(shortest(ACT1_DIR), shortest(ACT2_DIR))
    train_result = [a for t,l in zip(train_files, train_labels) for a in getPyramidHoFSegments(t, l, numFrames, LEVEL)]
    #test_result = [a for t,l in zip(test_files, test_labels) for a in getPyramidHoFSegments(t, l, numFrames, LEVEL)]
    train = [y for x in train_result[::2] for y in x]
    new_train_labels = np.concatenate(train_result[1::2])
    #test = [y for x in test_result[::2] for y in x]
    #new_test_labels = np.concatenate(test_result[1::2])
    train_threshold = int(max([len(x) for x in train]) * SEG_THRESHOLD)
    #test_threshold = int(max([len(x) for x in test]) * SEG_THRESHOLD)
    train_trimmed = [x for x,y in zip(train,new_train_labels) if len(x) > train_threshold]
    train_labels_trimmed = [y for x,y in zip(train,new_train_labels) if len(x) > train_threshold]
    #test_trimmed = [x for x,y in zip(test,new_test_labels) if len(x) > test_threshold]
    #test_labels_trimmed = [y for x,y in zip(test,new_test_labels) if len(x) > test_threshold]
    max_width = max(np.array([len(i) for i in test_trimmed]).max(),np.array([len(i) for i in test_trimmed]).max())
    train_pad = numpy_fillna(np.array(train_trimmed), max_width)
    #test_pad = numpy_fillna(np.array(test_trimmed), max_width)
    
    return train_pad, train_labels_trimmed#, test_pad, test_labels_trimmed

Now, we are ready to train and evaluate models. Define method for training models given featuresets

In [ ]:
def eval_model(classifier, (data, labels)):
    clf = None
    name = ""
    if (classifier == 0):
        clf = svm.SVC()
        name = "SVM"
    elif (classifier == 1):
        clf = tree.DecisionTreeClassifier()
        name = "Decision Tree"
    elif (classifier == 2):
        clf = ensemble.RandomForestClassifier()
        name = "Random Forest"
    else:
        clf = linear_model.LogisticRegression()
        name = "Logistic Regression"
    
    #clf.fit(train_features, train_labels)
    #predict = clf.predict(test_features)
    cross = cross_val_score(clf, data, labels, 5)
    
    return cross.mean(), cross.std(), name

def run(data):
    for i in range(4):
        score, name = eval_model(i, data);
        print(name + ": %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Test base HoF with padding

In [ ]:
data = getFeatures_Baseline(train_files, train_labels, test_files, test_labels)
run(data)

Test base HoF with trimming

In [ ]:
data = getFeatures_Trimmed(train_files, train_labels, test_files, test_labels)
run(data)

Test base HoF with segmentation

In [ ]:
data = getFeatures_Segmented(train_files, train_labels, test_files, test_labels)
run(data)

Test pyramid HoF with padding

In [ ]:
data = getFeatures_Pyramid_Baseline(train_files, train_labels, test_files, test_labels)
run(data)

test pyramid HoF with trimming

In [ ]:
data = getFeatures_Pyramid_Trimmed(train_files, train_labels, test_files, test_labels)
run(data)

test pyramid HoF with segmentation

In [ ]:
data = getFeatures_Pyramid_Segmented(train_files, train_labels, test_files, test_labels)
run(data)